In [1]:

import pandas as pd
import numpy as np
from bertopic import BERTopic
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import os
from collections import defaultdict
import cohere
import time

/media/4TB/elisa/AIxPA/env_aixpa/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load the model
mydir = './Results/'
model_folder =  "BERTopicModel/"
topic_model = BERTopic.load(mydir + model_folder)

In [3]:
# load data and repeat the same processing used when generating the model (see script 'modelgeneration.ipynb')

df = pd.read_csv("./FT/azioni.csv") # aggiustare il percorso del file CSV se necessario

df_filtered = df.dropna(subset=['descrizione']).copy()
df_filtered['text'] = (
    'Titolo: '
    + df_filtered['titolo'].fillna('').astype(str).str.strip()
    + '; Descrizione: '
    + df_filtered['descrizione'].astype(str).str.strip()
)

print("Totale documenti (titolo azione + descrizione) utilizzati per l'analisi:", len(df_filtered))
nltk.download('stopwords', quiet=True)

STOP_WORDS = set(stopwords.words('italian'))

def drop_stopwords(text: str) -> str:
    if not isinstance(text, str):
        return ""
    tokens = word_tokenize(text)
    return " ".join(w for w in tokens if w.lower() not in STOP_WORDS)

docs_unique = df_filtered['text'].fillna('').map(drop_stopwords).tolist()



Totale documenti (titolo azione + descrizione) utilizzati per l'analisi: 17024


In [4]:
# load the hierarchical topics (you can also recompute them with topic_model.hierarchical_topics(docs_unique) )
hierarchical_topics = pd.read_csv(mydir + "hierarchical_topics.csv", index_col=0)
res = topic_model.hierarchical_topics(docs_unique)


100%|██████████| 98/98 [00:00<00:00, 337.53it/s]


In [6]:
# this function cuts a BERTopic hierarchy DataFrame into clusters based on a distance threshold

def cut_bertopic_hierarchy(hier_df, threshold, exclude=(-1,)):
    # normalize expected column names (handles older hashes like '# ' prefixes)
    cols = {c.lower().replace('#','').strip(): c for c in hier_df.columns}
    P = cols.get('parent_id') or cols.get('parent')
    L = cols.get('child_left_id') or cols.get('left_child')
    R = cols.get('child_right_id') or cols.get('right_child')
    D = cols.get('distance') or cols.get('dist')

    # build child map: node -> (left, right, distance)
    child = {}
    for r in hier_df[[P, L, R, D]].itertuples(index=False, name=None):
        parent, left, right, dist = r
        child[parent] = (left, right, float(dist))

    # find the root (a parent that is never a child)
    all_nodes = set(hier_df[P]).union(hier_df[L]).union(hier_df[R])
    child_nodes = set(hier_df[L]).union(hier_df[R])
    roots = list(all_nodes - child_nodes)
    root = roots[0] if roots else None

    def leaves(node):
        if node not in child:           # leaf topic id
            return [node]
        l, r, _ = child[node]
        return leaves(l) + leaves(r)

    def collect(node):
        if node not in child:
            return [[node]]
        l, r, dist = child[node]
        if dist <= threshold:           # cut here → keep everything below as one cluster
            return [leaves(node)]
        return collect(l) + collect(r)

    clusters = collect(root) if root is not None else []
    # drop excluded topics and empty clusters
    clusters = [[int(t) for t in cl if t not in exclude] for cl in clusters]
    return [cl for cl in clusters if cl]

# version-safe unpacking
if isinstance(res, tuple):
    if len(res) == 2:
        hier_df, Z = res                    # DataFrame, linkage
    elif len(res) == 3:
        hier_df, Z, _ = res                 # ignore the 3rd
    else:
        hier_df, Z = res[0], None
else:
    hier_df, Z = res, None                  # only DataFrame returned

# cut the hierarchy at distance threshold 1, excluding outlier topic -1
clusters = cut_bertopic_hierarchy(hier_df, threshold=1, exclude=(-1,))
    

In [7]:
res = topic_model.hierarchical_topics(docs_unique)

# version-safe unpacking
if isinstance(res, tuple):
    if len(res) == 2:
        hier_df, Z = res                    # DataFrame, linkage
    elif len(res) == 3:
        hier_df, Z, _ = res                 # ignore the 3rd
    else:
        hier_df, Z = res[0], None
else:
    hier_df, Z = res, None       

100%|██████████| 98/98 [00:00<00:00, 345.64it/s]


In [8]:
# generate labels for the second-level clusters using an LLM (Cohere in this case)

df = pd.read_csv("./Results/topics_overview.tsv", sep='\t')
index = 0 
labels = []
f_out = open(mydir + "cluster_labels_second_level.txt", "w")
for k in range(len(clusters)):
    mystr = ''
    for m in range(len(clusters[k])):
        mystr = mystr+ (df.iloc[clusters[k][m]]['Label']+'\n')


    co = cohere.ClientV2(api_key="tavvrOAs6IOKOHk2XuH7mYSPG8PW0uoRkL4wPXZb")

    # prompt da passare all'LLM. In inglese per cohere perchè il modello funziona meglio in inglese, ma si può modificare il prompt a piacere (anche in italiano)
    system_message = f"""I seguenti topic sono stati raggruppati perché correlati:\n"
                "Obiettivo: proponi UNA sola etichetta breve e specifica (5–6 parole, solo sostantivi) "
                "che descriva al meglio l'insieme dei topic elencati."
                "Restituisci solo la label, senza spiegazioni né formattazioni. """
    prompt =f"""Topics:{mystr}"""
    
    index = index + 1
    res = co.chat(
        model="command-a-03-2025",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt},  

        ],
    )
    f_out.write('Etichetta cluster generata: \n')
    f_out.write(res.message.content[0].text)
    f_out.write('\nA partire dai seguenti topic:')
    f_out.write(mystr)
    f_out.write('\n')
    labels.append(res.message.content[0].text)

    if index %10==0: # ogni 10 richieste, pausa di 60 secondi per rispettare i limiti di frequenza dell'API
        time.sleep(60)
f_out.close()        


In [10]:
for k in range(len(clusters)):
    mystr = ''
    print(f"Macro categoria {k+1}: " + labels[k])
    for m in range(len(clusters[k])):
        mystr = mystr+ (df.iloc[clusters[k][m]]['Label']+', ')
    print("Sotto-categorie:   " + mystr)

Macro categoria 1: Servizi Comunitari e Educativi Locali
Sotto-categorie:   Gestione Impianti Sportivi, Gestione Spazi Comunitari, Attività Culturali e Spettacoli, Promozione Turistica e Culturale, Promozione della lettura infantile, Sostegno alla Genitorialità, Educazione Civica e Cittadinanza Attiva, Sostegno Attività Estive Minori, Attività Estive Giovanili, Sviluppo e Collaborazione Locale, Supporto e Promozione Familiare, Partecipazione Civica Attiva, Supporto Educativo Scolastico, Servizi educativi per l'infanzia, 
Macro categoria 2: Servizi Educativi e Welfare Estivo
Sotto-categorie:   Attività Estive per Bambini, Gestione Servizi Educativi Estivi, Politiche di Welfare Lavorativo, 
Macro categoria 3: Servizi educativi e tariffe asili nido
Sotto-categorie:   Servizi per l'infanzia, Gestione Tariffe Asili Nido, 
Macro categoria 4: Servizi di assistenza e cooperazione infantile
Sotto-categorie:   Servizi di assistenza all'infanzia, Servizi di Assistenza e Cooperazione, 
Macro categ